In [24]:
import requests
from bs4 import BeautifulSoup
from math import nan
import json, re, os

headers = {"user-agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/98.0.4758.82 Safari/537.36"}

def render_html(url):
    req = requests.get(url,headers=headers)
    page = BeautifulSoup(req.text,"html.parser")
    return page

def get_max(page):
    pagination = []
    for d in page.find("ul",class_="pagination").find_all("li"):
        try:
            d = int(d.get_text())
            pagination.append(d)
        except: pass
    return pagination[-1]

def get_urls(url,including=None):
    page = render_html(url)
    urls = []
    ref = "https://www.sbaflex.com"
    for section in page.find_all("section",class_="vacancy-card"):
        link = ref+section.find("a")['href']
        urls.append(link)
    urls = list(dict.fromkeys(urls))
    if including == "max_page":
        return urls, max_page
    else:
        return urls

In [31]:
# url = "https://www.sbaflex.com/pl_PL/vacatures"
# url = "https://www.sbaflex.com/pl_PL/vacatures-detail/vacancy/132-montaz-moskitier.html"
# page = render_html(url)

In [26]:
# scrape the initial
url = "https://www.sbaflex.com/pl_PL/vacatures"
urls, max_page = get_urls(url,including="max_page")
print("Page:",end=" ")

# scrape jobs
j = 0
for i in range(max_page):
    url = f"https://www.sbaflex.com/pl_PL/vacatures/offset/{j}"
    new_urls = get_urls(url)
    urls.extend(new_urls)
    print(j,end=",")
    j += 10
print("\b.")

Page: 0,10,20,30,40.


In [50]:
jobs = []
i = 0
print("Index:",end=" ")
for url in urls:
    print(i,end=",")
    page = render_html(url)
    try:
        h1 = page.find("section",class_="vacancies-detail").find("h1")
        title = h1.get_text().strip()
    except: title = nan
    items = {}
    for key in ["date","city","salary","hours"]:
        try: value = page.find("span",class_=key).get_text().strip()
        except: value = nan
        items.update({key:value})
    job = {
        'Link': url,
        'Active Form': items['date'],
        'Region': items['city'],
        'Wage': items['salary'],
        'Hours': items['hours'],
        'Title': title
    }
    jobs.append(job)
    i += 1
print("\b.")

Index: 0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50.


In [52]:
import pandas as pd

df = pd.DataFrame(jobs)
df.to_excel("sbaflex.xlsx",index=False,encoding="utf-8")